In [2]:
import numpy as np
import pandas as pd
import time
import os

# Custom Libraries -->
%load_ext autoreload
%autoreload 2
from src.tradier import Tradier

In [18]:
%load_ext dotenv
%dotenv
access_token = os.environ.get("access_token")

# 8 Random Stocks
SYMBOLS = ["TSLA", "AAPL", "AMZN", "GOOG"]

START_DATE = "2020-01-01"
END_DATE = "2023-12-31"

tradier = Tradier(access_token)

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [73]:
loadNewData = False
if loadNewData:
    dataframe = pd.DataFrame()

    for symbol in SYMBOLS:
        returns = tradier.getPriceChanges(symbol, START_DATE, END_DATE)

        dataframe[symbol] = returns['change']
        dataframe['date'] = returns['date']

        print("Fetched data for", symbol)
        time.sleep(3)

    display(dataframe)
    dataframe.to_csv('data/returns.csv', index=False)

Fetched data for TSLA
Fetched data for AAPL
Fetched data for AMZN
Fetched data for GOOG


,TSLA,date,AAPL,AMZN,GOOG
0,NaN,2020-01-02,NaN,NaN,NaN
1,0.037691,2020-01-03,0.003072,-0.005600,0.004704
2,-0.000068,2020-01-06,-0.011307,-0.002414,0.001588
3,0.047517,2020-01-07,0.020593,0.023925,0.035511
4,0.026658,2020-01-08,-0.008938,-0.003392,-0.004192
...,...,...,...,...,...
1001,0.019293,2023-12-22,-0.004691,0.003066,0.009661
1002,-0.008841,2023-12-26,-0.008044,-0.001366,0.005980
1003,0.015168,2023-12-27,-0.005785,0.000000,-0.001049
1004,0.020554,2023-12-28,0.008572,0.001042,-0.006861


In [83]:
data = pd.read_csv('data/returns.csv')
display(data.head(3))

,TSLA,date,AAPL,AMZN,GOOG
0,0.037691,2020-01-03,0.003072,-0.005600,0.004704
1,-0.000068,2020-01-06,-0.011307,-0.002414,0.001588
2,0.047517,2020-01-07,0.020593,0.023925,0.035511


In [90]:
is_asset = data.loc[:, data.columns != 'date']

mean_annual_returns = (1 + is_asset.mean())**252 - 1
covariance_matrix = is_asset.cov() * 252

# display(is_asset)
# print("Mean Annual Returns")
# print(mean_annual_returns)
# print("Covariance Matrix")
# print(covariance_matrix)

In [94]:
import plotly.graph_objects as go

# Optimal Allocation Optimization
NUMBER_OF_PORTFOLIOS = 1000
NUMBER_OF_ASSETS_IN_PORTFOLIO = 4

seed = np.random.randint(0, 1000)
np.random.seed(seed)

In [95]:
Markowitz = pd.DataFrame(columns=['Index', 'Symbols', 'Allocations', 'Returns', 'Volatility'])

for portfolio_index in range(NUMBER_OF_PORTFOLIOS):
    assets = np.random.choice(SYMBOLS, NUMBER_OF_ASSETS_IN_PORTFOLIO, replace=False)
    weights = np.random.rand(NUMBER_OF_ASSETS_IN_PORTFOLIO)

    # Ensure that weights sum to 1
    weights /= np.sum(weights)
    
    result = 0
    volatility = 0

    for i, asset in enumerate(assets):
        current_weight = weights[i] 
        result += current_weight * mean_annual_returns.loc[asset]
        
        for j, comparative_asset in enumerate(assets):
            volatility += weights[i] * weights[j] * covariance_matrix.loc[asset, comparative_asset]

    temp = pd.DataFrame({
        'Index': portfolio_index,
        'Symbols': ",".join(assets),
        'Allocations': str(weights.tolist()),
        'Returns': result,
        'Volatility': volatility,
    }, index=[0])

    Markowitz = pd.concat([Markowitz, temp])

In [100]:
risk_free_rate= 0.05

# Random Portfolio Allocations
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=Markowitz['Volatility'],
    y=Markowitz['Returns'],
    mode='markers',
    marker=dict(
        color=(Markowitz['Returns'] - risk_free_rate) / Markowitz['Volatility'],
        colorscale='RdBu',
        size=7,
        colorbar=dict(title="Sharp Ratio")
    ),
    text=Markowitz['Symbols'] + "<br>" + Markowitz['Allocations'],
))
fig.update_layout(
    template="plotly_white",
    title="Simulation of Random Portfolio Allocations",
    xaxis_title="Risk Volatility (Standard Deviation)",
    yaxis_title="Annualized Return",
)
fig.update_layout(coloraxis_colorbar=dict(title="Sharp Ratio"))

In [109]:
# Markowitz Efficient Frontier
from tqdm import tqdm

MarkowitzOptimized = pd.DataFrame(columns=['Index', 'Symbols', 'Allocations', 'Returns', 'Volatility'])

for i in tqdm(range(1000)):
    is_underpreforming_porfolio = False
    while True:
        assets = np.random.choice(SYMBOLS, NUMBER_OF_ASSETS_IN_PORTFOLIO, replace=False)
        weights = np.random.rand(NUMBER_OF_ASSETS_IN_PORTFOLIO)

        # Ensure that weights sum to 1
        weights /= np.sum(weights)
        
        result = 0
        volatility = 0

        for i, asset in enumerate(assets):
            current_weight = weights[i] 
            result += current_weight * mean_annual_returns.loc[asset]
            
            for j, comparative_asset in enumerate(assets):
                volatility += weights[i] * weights[j] * covariance_matrix.loc[asset, comparative_asset]

        for index, row in Markowitz.iterrows():
            if result < row['Returns'] and volatility > row['Volatility']:
                is_underpreforming_porfolio = True
                break
        
        if is_underpreforming_porfolio:
            break

        temp = pd.DataFrame({
            'Index': portfolio_index,
            'Symbols': ",".join(assets),
            'Allocations': str(weights.tolist()),
            'Returns': result,
            'Volatility': volatility,
        }, index=[0])

        MarkowitzOptimized = pd.concat([MarkowitzOptimized, temp])
        break

100%|██████████| 1000/1000 [00:12<00:00, 78.08it/s]


In [110]:
# Markowitz Efficient Frontier
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=MarkowitzOptimized['Volatility'],
    y=MarkowitzOptimized['Returns'],
    mode='markers',
    marker=dict(
        color=(MarkowitzOptimized['Returns'] - risk_free_rate) / MarkowitzOptimized['Volatility'],
        colorscale='RdBu',
        size=7,
        colorbar=dict(title="Sharp Ratio")
    ),
    text=MarkowitzOptimized['Symbols'] + "<br>" + MarkowitzOptimized['Allocations'],
))
fig.update_layout(
    template="plotly_white",
    title="Markowitz Efficient Frontier",
    xaxis_title="Risk Volatility (Standard Deviation)",
    yaxis_title="Annualized Return",
)
fig.update_layout(coloraxis_colorbar=dict(title="Sharp Ratio"))